In [1]:
! pip install pymongo

**Setting up connection with MongoDB**

In [3]:
import pymongo
import time

In [4]:
# Setting up timer
t00 = time.time() 

client = pymongo.MongoClient('mongodb://localhost:27017')
my_db = client["420Project"]
firearms_info = my_db.firearms

# closing timer
t01 = time.time()
print("Time elapsed is",t01-t00)

Time elapsed is 0.023524999618530273


In [27]:
# To check first record in the collection
firearms_info.find_one()

{'_id': ObjectId('64dc77237dee8e6cbc7ea572'),
 'month': '2023-07',
 'state': 'Alabama',
 'permit': 10197,
 'permit_recheck': 96,
 'handgun': 15324,
 'long_gun': 9081,
 'other': 1285,
 'multiple': 877,
 'admin': 0,
 'prepawn_handgun': 21,
 'prepawn_long_gun': 3,
 'prepawn_other': 2,
 'redemption_handgun': 2078,
 'redemption_long_gun': 788,
 'redemption_other': 23,
 'returned_handgun': 75,
 'returned_long_gun': 0,
 'returned_other': 0,
 'rentals_handgun': 0,
 'rentals_long_gun': 0,
 'private_sale_handgun': 32,
 'private_sale_long_gun': 21,
 'private_sale_other': 7,
 'return_to_seller_handgun': 0,
 'return_to_seller_long_gun': 0,
 'return_to_seller_other': 0,
 'totals': 39910}

**First query: To find state with maximum no. of background checks initiated for handgun in year 2022**

In [5]:
# Setting up timer
t11 = time.time()

months_2022 = ['2022-01','2022-02','2022-03','2022-04','2022-05','2022-06','2022-07','2022-08','2022-09','2022-10','2022-11','2022-12']

agg_result1 = firearms_info.aggregate([
    {"$match": {'month':{'$in':months_2022}}},
    {
        "$group" :{
            "_id":'$state',
            "TotalHandguns":{
                "$sum":{ "$add": ["$handgun","$prepawn_handgun","$private_sale_handgun","$returned_handgun","$rentals_handgun","$return_to_seller_handgun"]}
            }    
        }   
    }   
    ])

for record in agg_result1:
    print(record)
# closing timer
t12 = time.time()
print("Time elapsed is",t12-t11)

{'_id': 'New Jersey', 'TotalHandguns': 101828}
{'_id': 'Virginia', 'TotalHandguns': 323883}
{'_id': 'Alabama', 'TotalHandguns': 250476}
{'_id': 'Ohio', 'TotalHandguns': 317595}
{'_id': 'Rhode Island', 'TotalHandguns': 15100}
{'_id': 'Missouri', 'TotalHandguns': 255869}
{'_id': 'Hawaii', 'TotalHandguns': 21}
{'_id': 'South Carolina', 'TotalHandguns': 151454}
{'_id': 'Washington', 'TotalHandguns': 288839}
{'_id': 'Mississippi', 'TotalHandguns': 121245}
{'_id': 'Arkansas', 'TotalHandguns': 91701}
{'_id': 'Indiana', 'TotalHandguns': 286140}
{'_id': 'Alaska', 'TotalHandguns': 38511}
{'_id': 'Kentucky', 'TotalHandguns': 139959}
{'_id': 'New Mexico', 'TotalHandguns': 87625}
{'_id': 'Puerto Rico', 'TotalHandguns': 69829}
{'_id': 'Maryland', 'TotalHandguns': 118822}
{'_id': 'Connecticut', 'TotalHandguns': 76208}
{'_id': 'Idaho', 'TotalHandguns': 77504}
{'_id': 'New York', 'TotalHandguns': 131190}
{'_id': 'Arizona', 'TotalHandguns': 241552}
{'_id': 'North Dakota', 'TotalHandguns': 23577}
{'_id':

**Second query: Under redemption, Which category has been requested for maximum number of background checks across 5 years**

In [6]:
# Setting up timer
t21 = time.time()

agg_result2 = firearms_info.aggregate([

    {
        "$group" :{
            "_id":'null',
            "max_redemption_handgun":{
                "$max":"$redemption_handgun"
            },
            "max_redemption_longgun":{
                "$max":"$redemption_long_gun"
            },
            "max_redemption_other":{
                "$max":"$redemption_other"
            }
            
        }   
    }   
    ])

for record in agg_result2:
    print(record)

# closing timer
t12 = time.time()
print("Time elapsed is",t12-t11)

{'_id': 'null', 'max_redemption_handgun': 9486, 'max_redemption_longgun': 5047, 'max_redemption_other': 104}
Time elapsed is 5.371674537658691


**Third query: ‘Out of total which category of Background checks have majority total of requests in July 2022 which category and state’**

In [7]:
# Setting up timer
t31 = time.time()


agg_result3 = firearms_info.aggregate([
    {"$match": {'month':'2022-07'}},
    {
        "$group" :{
            "_id":'null',
            "sum_permit":{
                "$sum":"$permit"
            },
            "sum_permit_recheck":{
                "$sum":"$permit_recheck"
            },
            "sum_handgun":{
                "$sum":"$handgun"
            },
            "sum_long_gun":{
                "$sum":"$long_gun"
            },
            "sum_other":{
                "$sum":"$other"
            },
            "sum_multiple":{
                "$sum":"$multiple"
            },
            "sum_admin":{
                "$sum":"$admin"
            },
            
            "TotalPrepawn":{
                "$sum":{ "$add": ["$prepawn_handgun","$prepawn_long_gun","$prepawn_other"]}
            },
            "TotalRedemption":{
                "$sum":{ "$add": ["$redemption_handgun","$redemption_long_gun","$redemption_other"]}
            },
            "TotalReturned":{
                "$sum":{ "$add": ["$returned_handgun","$returned_long_gun","$returned_other"]}
            },
            "TotalRental":{
                "$sum":{ "$add": ["$rentals_handgun","$rentals_long_gun"]}
            },
            "TotalPrivateSale":{
                "$sum":{ "$add": ["$private_sale_handgun","$private_sale_long_gun","$private_sale_other"]}
            },
            "TotalReturnToSeller":{
                "$sum":{ "$add": ["$return_to_seller_handgun","$return_to_seller_long_gun","$return_to_seller_other"]}
            }
        }   
    }   
    ])


for record in agg_result3:
    print(record)

# closing timer
t12 = time.time()
print("Time elapsed is",t12-t11)

{'_id': 'null', 'sum_permit': 647869, 'sum_permit_recheck': 477166, 'sum_handgun': 688408, 'sum_long_gun': 380096, 'sum_other': 60442, 'sum_multiple': 23967, 'sum_admin': 438, 'TotalPrepawn': 370, 'TotalRedemption': 50764, 'TotalReturned': 10266, 'TotalRental': 5, 'TotalPrivateSale': 18187, 'TotalReturnToSeller': 172}
Time elapsed is 27.262148141860962
